In [1]:
# scrape the data from  http://www.start.umd.edu/gtd/ and save in Excel Workbook
 
import urllib
from bs4 import *
import numpy as np
import pandas as pd 
import xlsxwriter
import re
from multiprocessing.dummy import Pool as ThreadPool


# returns the list of url in website

def url_links_list():
    url_list = []
    url_head = 'http://www.start.umd.edu/gtd/search/Results.aspx?page='
    url_tail = '&casualties_type=b&casualties_max=&start_yearonly=1970&end_yearonly=2015&dtp2=all&weapon=1,2,6,7,5,8,9,4,12,3,11,13,10&charttype=line&chart=overtime&ob=GTDID&od=desc&expanded=yes#results-table'
    page_num = range(1,7800)
    for pg_num in page_num:
        url = url_head + str(pg_num) + url_tail
        url_list.append(url)
    return url_list

# returns lists of data from the list of urls 

def fetch_Data(x): 
    big_list = []
    fhand = urllib.urlopen(x).read()
    soup = BeautifulSoup(fhand, 'html.parser')
    for row in soup.find_all('tr'):
        col_mark = 0
        lis = []
        td_values = row.find_all('td')
        for col in td_values:
            lis.insert(col_mark, col.get_text())
            col_mark+= 1
        big_list.append(lis)
    return big_list

# Write data_list to Excel Workbook

def write_to_excel(data_set):
    workbook = xlsxwriter.Workbook('data_GTP_tables.xlsx')
    worksheet = workbook.add_worksheet()
    row1 = 0
    for first_list in data_set:
        for row_val in first_list:
            col1 = 0
            for column in row_val:
                worksheet.write(row1, col1, column)
                col1 += 1
            row1 += 1
    workbook.close() 
    
  


pool = ThreadPool(4)
results = pool.map(fetch_Data, url_links_list())

pool.close() 
pool.join()

if __name__ == '__main__':
    write_to_excel(results)
